In [1]:
### 필요 라이브러리
import os
import pandas as pd
import numpy as np
import datetime as dt
import math
import warnings
warnings.filterwarnings(action='ignore')
# save numpy array as npy file
from numpy import asarray
from numpy import save
from DataInfo import *
### 데이터 보여지는 개수 설정
pd.set_option('display.max_columns', 40)
pd.set_option('display.max_rows', 300)

In [2]:
mbrnlist= MBRN_sum_top[:15]

In [3]:
path_dir = '/Data/ksqord'
file_list=os.listdir(path_dir)
file_list.sort()

In [4]:
columnlist=['MBR_NO','BRN_NO','ISU_CD','ASKBID_TP_CD','MODCANCL_TP_CD','PT_TP_CD']
featlist=columnlist + ['ORD_QTY','ORD_PRC']
filename = file_list[0]
print("extracting stats from", filename )
df = pd.read_csv(path_dir+'/'+filename, sep=',', names=header_df, encoding="cp949")
df['MBRN'] = list(zip(df['MBR_NO'],df['BRN_NO']))
df = df[df['MBRN'].isin(mbrnlist) ]
df = df[featlist]
df.fillna(0)
df['ORD_VOL'] = df['ORD_QTY'] * df['ORD_PRC']
gdf = df.groupby(columnlist).sum()

extracting stats from ksqord_20170601.csv


In [5]:
for filename in file_list[1:3]:
    print("extracting stats from", filename )
    df = pd.read_csv(path_dir+'/'+filename, sep=',', names=header_df, encoding="cp949")
    df['MBRN'] = list(zip(df['MBR_NO'],df['BRN_NO']))
    df = df[df['MBRN'].isin(mbrnlist) ]
    df = df[featlist]
    df.fillna(0)
    df['ORD_VOL'] = df['ORD_QTY'] * df['ORD_PRC']
    tgdf = df[featlist].groupby(columnlist).sum()
    gdf = gdf.add(tgdf).fillna(gdf).fillna(tgdf)
    print(gdf)

extracting stats from ksqord_20170602.csv
                                                                 ORD_PRC  \
MBR_NO BRN_NO ISU_CD       ASKBID_TP_CD MODCANCL_TP_CD PT_TP_CD            
2      83     HK0000054723 1            1              0         40620.0   
                           2            1              0          6335.0   
                                        3              0          3145.0   
              HK0000055043 1            1              0          5950.0   
                                        2              0          1175.0   
...                                                                  ...   
56     17990  KYG3931T1076 2            3              0          2800.0   
              USU652221081 1            1              0         60775.0   
                                        3              0          4985.0   
                           2            1              0         29805.0   
                                        3     

In [29]:
gdf.to_csv('./StatsForDataSelection/AlmostFullColumn_ISU'+"MBRN_sum_top")

In [2]:
tgdf = pd.read_csv('./StatsForDataSelection/AlmostFullColumn_ISU'+"MBRN_sum_top")

In [3]:
tgdf

,MBR_NO,BRN_NO,ISU_CD,ASKBID_TP_CD,MODCANCL_TP_CD,PT_TP_CD,ORD_PRC,ORD_QTY,ORD_VOL
0,2,83,HK0000054723,1,1,0,1201306.0,2538055.0,246864450.0
1,2,83,HK0000054723,1,2,0,142999.0,336761.0,NaN
2,2,83,HK0000054723,1,3,0,278599.0,898955.0,NaN
3,2,83,HK0000054723,2,1,0,1926083.0,1817786.0,15725.0
4,2,83,HK0000054723,2,2,0,173995.0,233358.0,NaN
...,...,...,...,...,...,...,...,...,...
110173,56,17990,USU652221081,1,2,0,285460.0,173767.0,NaN
110174,56,17990,USU652221081,1,3,0,2302840.0,308574.0,NaN
110175,56,17990,USU652221081,2,1,0,4894725.0,1337455.0,9049550.0
110176,56,17990,USU652221081,2,2,0,239145.0,175282.0,NaN


In [5]:
tgdf= tgdf.reset_index()
tgdf = tgdf.pivot(index= ['MBR_NO', 'BRN_NO', 'ISU_CD','ASKBID_TP_CD','PT_TP_CD'],columns=['MODCANCL_TP_CD'],values='ORD_VOL')

In [6]:
# tgdf = tgdf.reset_index()
tgdf = tgdf.rename(columns={1: 'NEW', 2:'EDIT',3:'CANCL'})

In [7]:
tgdf=tgdf.fillna(0)

In [8]:
tgdf['NET']= tgdf['NEW']-tgdf['CANCL']

In [9]:
tgdf

MODCANCL_TP_CD                                            NEW        EDIT  \
MBR_NO BRN_NO ISU_CD       ASKBID_TP_CD PT_TP_CD                            
2      83     HK0000054723 1            0         246864450.0         0.0   
                           2            0             15725.0         0.0   
              HK0000055043 1            0           6859270.0   1990450.0   
                           2            0             40120.0         0.0   
              HK0000057197 1            0          16544019.0    209864.0   
...                                                       ...         ...   
56     17990  KYG2115T1076 2            0          70663340.0  11089085.0   
              KYG3931T1076 1            0           9051000.0         0.0   
                           2            0           1165000.0         0.0   
              USU652221081 1            0          24017045.0         0.0   
                           2            0           9049550.0         0.0   

MODCANCL_TP_CD                                         CANCL          NET  
MBR_NO BRN_NO ISU_CD       ASKBID_TP_CD PT_TP_CD                           
2      83     HK0000054723 1            0                0.0  246864450.0  
                           2            0            15725.0          0.0  
              HK0000055043 1            0           708000.0    6151270.0  
                           2            0                0.0      40120.0  
              HK0000057197 1            0          2959817.0   13584202.0  
...                                                      ...          ...  
56     17990  KYG2115T1076 2            0         44252545.0   26410795.0  
              KYG3931T1076 1            0                0.0    9051000.0  
                           2            0          1165000.0          0.0  
              USU652221081 1            0                0.0   24017045.0  
                           2            0          3159480.0    5890070.0  

[37860 rows x 4 columns]

In [10]:
tgdf.sum(level=[2]).sort_values('NEW',ascending=False).index[:100]

Index(['KR7223310004', 'KR7161580006', 'KR7060260007', 'KR7024840001',
       'KR7035720002', 'KR7049180003', 'KR7217270008', 'KR7145020004',
       'KR7083660001', 'KR7031860000', 'KR7030270003', 'KR7192080000',
       'KR7197210008', 'KR7068270008', 'KR7014940001', 'KR7053030003',
       'KR7201490000', 'KR7086900008', 'KR7086390002', 'KR7025440009',
       'KR7064260003', 'KR7036930006', 'KR7067170001', 'KR7033830001',
       'KR7068790005', 'KR7065060006', 'KR7102940004', 'KR7091590000',
       'KR7035480003', 'KR7008800005', 'KR7050110006', 'KR7114570005',
       'KR7036690006', 'KR7240810002', 'KR7178920005', 'KR7130960008',
       'KR7034230003', 'KR7036830008', 'KR7051370005', 'KR7043580000',
       'KR7090360009', 'KR7046890000', 'KR7204840003', 'KR7131370009',
       'KR7056190002', 'KR7001840008', 'KR7224060004', 'KR7033500000',
       'KR7078150000', 'KR7090710005', 'KR7067390005', 'KR7064240005',
       'KR7108790007', 'KR7212560007', 'KR7006920003', 'KR7049120009',
      